# Работа со строковыми значениями

In [622]:
import re
import nltk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [623]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

In [624]:
for k,v in obj.items():
    print(f'{k:<9} = {v}')

home_page = https://github.com/pypa/sampleproject
keywords  = sample setuptools development
license   = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [625]:
obj = "Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"

for i in obj:
    print('ПМ' + re.search(r'\d\d-\d', i).group(0))

ПМ19-1
ПМ20-4
ПМ20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [626]:
data = r'Написать регулярное выражение,которое позволит найти номера групп студентов.'

re.split('\W', data, maxsplit=0, flags=0)  # \W - любой символ альтернатива [ .,]

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов',
 '']

4. Найдите в тексте все последовательности: "вол, воз, вон".
 
Текст: "вол воБ во8 воз вок вог во4 воХ во! воь вон" (От Калажоков З.Х.)

In [627]:
data = "вол, воз, вон. вол воБ во8 воз вок вог во4 воХ во! воь вон"

re.findall(r'во[злн]', data)

['вол', 'воз', 'вон', 'вол', 'воз', 'вон']

5. Напишите регулярное выражение, которое найдёт все кабинеты с трёхзначным номером: 100 - 999 в строке 

'147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'. 

Шаблон кабинета: ddd кабинет, где d - арабская цифра. (От Калажоков З.Х.)

In [628]:
data = '147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'

re.findall(r'[1-9]\d{2} кабинет', data)

['147 кабинет', '843 кабинет', '514 кабинет', '246 кабинет', '572 кабинет']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [629]:
a = [str(rnd[[rnd.columns[0], rnd.columns[1]]].values.max())]
a.extend(rnd.columns)
max_len = len(max(a, key=len)) + 4

In [630]:
obj = pd.read_csv('labs/recipes_sample.csv')
rnd = obj.sample(5)[['id', 'minutes']]

print(f'|{rnd.columns[0]:^{max_len}}|{rnd.columns[1]:^{max_len}}|')
print('|' + '-'*(max_len*2 +len(rnd.columns)-1) + '|')
for i,j in zip(rnd[rnd.columns[0]],rnd[rnd.columns[1]]):
    print(f'|{i:^{max_len}}|{j:^{max_len}}|')

|    id     |  minutes  |
|-----------------------|
|  295541   |    60     |
|   81593   |    50     |
|  159118   |    60     |
|  256544   |    35     |
|  391593   |    35     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [631]:
recipes = pd.read_csv('labs/recipes_sample.csv')
with open('labs/steps_sample.xml') as f:
    steps = BeautifulSoup(f, 'xml')

In [632]:
data = dict()
for recipe in steps.find_all('recipe'):
    idr = recipe.find('id').next
    steps = [i.next for i in recipe.steps.find_all('step')]
    data[idr] = steps
data

{'44123': ['in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes',
  'add thyme , oregano and basil',
  'saute 5 minutes more',
  'add wine and deglaze pan',
  'add hot chicken stock and reduce by one-third',
  'add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets',
  'simmer 5 minutes',
  'add cream , simmer 5 minutes more and season to taste',
  'drop in remaining butter , piece by piece , stirring until melted and serve immediately',
  'smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare',
  'chef meskan uses applewood chips and does not allow the grill to become too hot'],
 '67664': ['mix all the ingredients using a blender',
  'pour into popsicle molds',
  'freeze and enjoy !'],
 '38798': ['combine all ingredients in a large bowl and mix well',
  'shape into one-inch balls',
  'cover and refrigerate or freeze until ready to bake',
  'preheat oven to 350 degrees',
  'place on ungr

In [633]:
def show_info(name, steps, minutes, author_id):
    a_str = f'"{name.title()}"\n\n'
    a_str += ''.join([f'{i+1}. {steps[i][0].upper() + steps[i][1:]}\n' for i in range(len(steps))])
    a_str += f'----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'
    
    print(a_str)
    return a_str

In [634]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

"George S At The Cove Black Bean Soup"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
----------
Автор: 35193
Среднее время приготовления: 90 минут



In [635]:
show_info(recipes[recipes.id == 170895].name.values[0], data['170895'], recipes[recipes.id == 170895].minutes.values[0], recipes[recipes.id == 170895].contributor_id.values[0])

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

'"Leeks And Parsnips  Sauteed Or Creamed"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n4. Heat\n5. Add the garlic and fry \'til fragrant\n6. Add leeks and fry until the leeks are tender , about 6-minutes\n7. Meanwhile , peel and chunk the parsnips into one-inch pieces\n8. Place in a steaming basket and steam \'til they are as tender as you prefer\n9. I like them fork-tender\n10. Drain parsnips and add to the skillet with the leeks\n11. Add salt and pepper\n12. Gently sautee together for 5-minutes\n13. At this point you can serve it , or continue on and cream it:\n14. In a jar with a screw top , add the half-n-half and arrowroot\n15. Shake \'til blended\n16. Turn heat to low under the leeks and parsnips\n17. Pour in the arrowroot mixture , stirring gently as you pour\n18. If too thick , gradually add the water\n19. Let simmer for a couple of minutes\n20. Taste to adjust sea

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [636]:
' '.join(data['25082'])

"proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form stir in the salt and oil add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board the dough should be springy and nice to work with when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a plastic bag , and maybe a plate let the dough rise until it springs back when you

In [637]:
re.findall(r'\d+ minute[s ]|\d+ hour[s ]', ' '.join(data['25082']), flags=0)

['20 minutes',
 '10 minutes',
 '2 hours',
 '10 minutes',
 '20 minutes',
 '30 minutes']

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [638]:
fs = 'this asd asd da3ewa, but'

def check(st):
    return [bool(re.match(r'this[A-Za-z0-9_ ]+, {0,1}but', i, flags=0)) if str(i) != 'nan' else False for i in st]
    

fnd = recipes[check(recipes.description)]
fnd

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
76,neezy peazy lunch,24760,35,29291,2002-04-08,NaN,"this is a great meal eaten the same day ,but e...",6.0
183,3 pepper quiche,311313,75,186802,2008-07-02,NaN,this was adapted from a recipe i found on the ...,13.0
337,addictive chex mix,277562,21,260464,2008-01-08,NaN,this is kind of similar to some of the other v...,12.0
486,almas pite hungarian apple cake,200227,1500,388615,2006-12-11,18.0,"this is a moist, buttery apple cake. the doug...",10.0
678,amish tears on your pillow pie,40858,75,6258,2002-09-20,NaN,this pie does not have very many ingredients i...,6.0
...,...,...,...,...,...,...,...,...
29315,white spinach lasagna,92419,85,101034,2004-06-01,12.0,this is so simple to put together and tastes s...,11.0
29340,whole rump barbecued,131322,1455,214855,2005-07-27,7.0,"this is so good, but does require 24 hours mar...",NaN
29853,zesty salad with tortilla strips,170416,9,169969,2006-05-30,10.0,this simple salad pairs wonderfully with any m...,11.0
29967,zucchini spice cake,80067,65,4439,2004-01-02,11.0,"this recipe is pretty close to bizzare, but it...",4.0


In [639]:
len(fnd)

133

In [640]:
print('\n\n'.join(fnd.description.values[:3]))

this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [641]:
for i in range(len(data['72367'])):
    data['72367'][i] = re.sub(r'. / .', lambda m: m.group(0).replace(' ', ''),  data['72367'][i])

In [642]:
data['72367']

['mix butter , flour , 1/3 c',
 'sugar and 1-1/4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1/4 c',
 'sugar , eggs and 1/2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1/3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [330]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/admin/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package name

True

In [643]:
string = (' '.join([' '.join(i) for i in data.values()])).lower()

In [644]:
tokenize = np.unique(np.array(nltk.word_tokenize(string, language='russian')))
tokenize, len(tokenize)

(array(['!', '#', '$', ..., '~~important', '~~~~~~or', '~~~~~~~~~~~~~~'],
       dtype='<U35'),
 19568)

In [645]:
tokenize = tokenize[list(map(lambda x: x.isalpha(), tokenize))]
tokenize, len(tokenize)

(array(['a', 'aaahhhh', 'aand', ..., 'zukes', 'zup', 'zwieback'],
       dtype='<U35'),
 14926)

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [646]:
tk = list(map(nltk.sent_tokenize, recipes.description.fillna('').values))
tk[:2]

[["an original recipe created by chef scott meskan, george's at the cove.",
  'we enjoyed this when we visited this restaurant in la jolla, california.',
  'this recipe is requested so often, they have it printed and ready at the hostess stand.',
  "it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!"],
 ['my children and their friends ask for my homemade popsicles morning, noon and night.',
  'i never turn them down; who am i to tell them that they are good for them!',
  'for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.']]

In [647]:
tkl = np.argsort(np.array(list(map(len, tk))))[:-6:-1]
tkl

array([18408,   481, 22566,  6779, 16296])

In [648]:
recipes.iloc[tkl]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [649]:
lst = nltk.pos_tag(recipes[recipes.id == 241106].name.values[0].split())
lst

[('eggplant', 'JJ'),
 ('steaks', 'NNS'),
 ('with', 'IN'),
 ('chickpeas', 'NNS'),
 ('feta', 'VBP'),
 ('cheese', 'JJ'),
 ('and', 'CC'),
 ('black', 'JJ'),
 ('olives', 'NNS')]

In [650]:
a,b = '', ''
for k,v in lst:
    a += f'{v:^{len(k)+1}}'
    b += f' {k}'

print(f'{a}\n{b}')

   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
 eggplant steaks with chickpeas feta cheese and black olives
